# Preprocesamiento de Texto para Clasificación Multi-Etiqueta de StackOverflow

Este proyecto tiene como objetivo preprocesar texto y construir features para clasificación multi-etiqueta de posts de StackOverflow. Debido a la longitud del proyecto, está dividido en dos partes. En esta primera parte nos enfocaremos en el preprocesado y construcción de features.

Este proyecto tiene como objetivo predecir etiquetas de *posts* de [StackOverflow](https://stackoverflow.com). Técnicamente, es una tarea de clasificación multi-etiqueta. Nótese que el lenguaje en el que están escritas las entradas es el **INGLÉS**, con lo que algunos de los pasos son específicos para dicho idioma.

Debido a la longitud del proyecto, está dividido en dos partes. En esta primera parte nos enfocaremos en el preprocesado y construcción de features. Para terminar entrenando modelos en una segunda parte.

## Librerías

Haremos uso de las siguientes librerías
- [Numpy](http://www.numpy.org)
- [Pandas](https://pandas.pydata.org)
- [scikit-learn](http://scikit-learn.org/stable/index.html)
- [NLTK](http://www.nltk.org) — librería básica para trabajar con texto en Python

aunque si quieres pudes usar spaCy para algunas tareas.

##  Preprocesado

Una de las primeras técnicas que vamos a utilizar para preprocesar textos es la eliminación de las conocidas como **stop words**, es decir, palabras que no aportan mucho significado, pero que son necesarias para que el texto sea legible y siga las normas. Para ello, lo primero es conseguir una lista con las *stop words* del lenguaje requerido.

Una opción para conseguir esta lista de palabras, es usar la librería `nltk`.

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/odremanferrer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


En este proyecto tenemos un dataset con títulos de entradas de StackOverflow, debidamente etiquetado (con 100 etiquetas distintas).

In [17]:
from __future__ import annotations
from ast import literal_eval
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from string import ascii_lowercase
import re

In [9]:
def read_data(filename):
    data = pd.read_csv(filename, sep='\t')
    data['tags'] = data['tags'].apply(literal_eval)
    return data

In [10]:
train = read_data('data/train.tsv')
train, validation = train_test_split(train, test_size = .15, random_state = 0)
test = read_data('data/test.tsv')

In [11]:
train.head()

,title,tags
96598,How to create an array of leaf nodes of an htm...,"[javascript, arrays, dom]"
10007,how to make maven use test resources,"[java, maven]"
15802,How do I get the path where the user installed...,[java]
9114,why are my buttons not showing up?,"[java, swing]"
34247,How to loop an array with strings as indexes i...,"[php, arrays, string, loops]"


Como vemos, la columna `"title"` contiene los títulos de las entradas, y la columna `"tags"` una lista con las etiquetas de cada entrada, que puede ser un número arbitrario.

Para seguir los convenios, inicializamos `X_train`, `X_val`, `X_test`, `y_train`, `y_val`, `y_test`.

In [12]:
X_train, y_train = train['title'].values, train['tags'].values
X_val, y_val = validation['title'].values, validation['tags'].values
X_test, y_test = test['title'].values, test['tags'].values

La principal dificultad de trabajar con lenguaje natural es que no está estructurado. Si cojemos el texto y creamos tokens simplemente separando por los espacios, tendremos *tokens* como `'3.5?'`, `'do.'`, etc. Para evitar esos problemas, es útil preprocesar el texto.

### **Tarea 1 (Preprocesado):**

Implementa la función `text_tokenizer()` y `text_prepare()` siguiendo las instrucciones.

In [15]:
def text_tokenizer(text :str) -> list[str]:
    """
    Transforma un texto (str) en una lista de palabras/tokens (list).
    Es importante usar esta función siempre para ser consistentes.
    """
    ## ESCRIBE AQUÍ TU CÓDIGO
    if not text:  # Si text es None o vacío
        return []
    return text.split()
    ##


# cargamos estas variables fuera de la función
# ya que la creación del set de stopwords es costoso y no queremos
# que se repita cada vez que se llame a la función
REPLACE_BY_SPACE = "[/(){}\[\]\|@,;]"
GOOD_CHARS = ascii_lowercase + "".join([str(n) for n in range(10)]) + " #+_"
STOPWORDS = set(stopwords.words("english"))


def text_prepare(text:str) -> str:
    """
    Preprocesa el texto inicial:
    1. eliminando espacios al inicio y final, y convirtiéndolo a minúsculas
    2. cambia los caracteres de REPLACE_BY_SPACE por espacios
    3. elimina los caracteres que no estén en GOOD_CHARS
    4. elimina los tokens que sean STOPWORDS

    text: str
    return: str
    """
    ## ESCRIBE AQUÍ TU CÓDIGO
    if not text:  # Si text es None o vacío
        return ""
        
    # 1. Eliminar espacios y convertir a minúsculas
    text = text.lower().strip()
    
    # 2. Reemplazar caracteres especiales por espacios
    text = re.sub(REPLACE_BY_SPACE, ' ', text)
    
    # 3. Eliminar caracteres que no estén en GOOD_CHARS
    text = ''.join([char for char in text if char in GOOD_CHARS])
    
    # 4. Eliminar stopwords
    tokens = text_tokenizer(text)
    tokens = [token for token in tokens if token not in STOPWORDS]
    
    return ' '.join(tokens)
    ##

In [18]:
def test_text_prepare():
    examples = ["   SQL Server - any equivalent of Excel's CHOOSE function?",
                "How to free c++ memory vector<int> * arr?"]
    answers = ["sql server equivalent excels choose function",
               "free c++ memory vectorint arr"]
    for ex, ans in zip(examples, answers):
        if text_prepare(ex) != ans:
            return "Respuesta incorrecta para: '%s'" % text_prepare(ex)
    return '¡Tests correctos!'

print(test_text_prepare())

¡Tests correctos!


Ahora preprocesamos los textos de todos los conjuntos:

In [19]:
X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

In [20]:
X_train[:3]

['create array leaf nodes html dom using javascript',
 'make maven use test resources',
 'get path user installed java application']

### **Tarea 2 (Cuentas de palabras y etiquetas):**

Cuénta cuantas veces aparece cada token (palabra) y cada etiqueta en el corpus de entrenamiento. Es decir, crea un diccionario con las cuentas totales de palabras y etiquetas.

El resultado deben ser dos diccionarios *tags_counts* y *words_counts* del tipo `{'palabra_o_etiqueta': cuentas}`.

In [29]:
# Diccionario con todas las etiquetas del corpus de entrenamiento con sus cuentas
tags_counts = {}
for tags_list in y_train:
    for tag in tags_list:
        tags_counts[tag] = tags_counts.get(tag, 0) + 1

# Diccionario con todas las palabras del corpus de entrenamiento con sus cuentas
words_counts = {}
for text in X_train:
    for word in text_tokenizer(text):
        words_counts[word] = words_counts.get(word, 0) + 1

Exploramos las más comunes:

In [22]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:3]
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]
print(most_common_tags)
print(most_common_words)

[('c#', 16259), ('javascript', 16219), ('java', 15835)]
[('using', 7000), ('php', 4774), ('java', 4683)]


### Transformando el texto a vectores

Vamos a construir los vectores asociados a cada frase en dos representaciones distintas, Bag of Words y tf-idf. Dejaremos la segunda para la parte 2 del proyecto.


#### Bag of words

Recuerda que para crear la representación de *bag of words*, convertimos cada frase en un vector que cuenta el número de ocurrencias de cada token. Se siguien los pasos:
1. Encuentra los **N** tokens mas comunes del corpus de entrenamiento y se les asigna un índice, este es nuestro **vocabulario**. Creamos un diccionario para convertir de tokens a índices y viceversa.
2. Para cada frase en el corpus, creamos un vector de dimensión **N** y lo inicializamos con ceros.
3. Iteramos sobre los tokens de cada frase, y si el token está en el diccionario, incrementamos en 1 el índice correspondiente del vector.
   
**Tarea 3 (BagOfWords):**

Contruye la función que transforma un texto en su representación *bag of words*.

Implementa la codificación de *bag of words* en la función `my_bag_of_words()` con un tamaño de diccionario de **N=5000**. Para definir el diccionario, sólo podemos usar el conjunto de entrenamiento, sino tendríamos un *data leaking*.

Primero, contruimos el vocabulario y los diccionarios correspondientes, así como un `set` con las palabras del diccionario.

In [30]:
DICT_SIZE = 5000

most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:DICT_SIZE]

# Crear los diccionarios de mapeo
WORDS_TO_INDEX = {word: idx for idx, (word, count) in enumerate(most_common_words)}
INDEX_TO_WORDS = {idx: word for word, idx in WORDS_TO_INDEX.items()}
##

In [31]:
def my_bag_of_words(text: str, words_to_index: dict[str, int]) -> np.array:
    """
    text: str
    words_to_index: dict, diccionario con los índices del vocabulario

    return
    result_vector: numpy.array, vector con la representación bag-of-words de `text`
    """
    dict_size = len(words_to_index)
    result_vector = np.zeros(dict_size)

    tokens = text_tokenizer(text)
    
    # Contar ocurrencias de cada palabra que esté en el vocabulario
    for token in tokens:
        if token in words_to_index:
            result_vector[words_to_index[token]] += 1
    ##

    return result_vector

In [32]:
def test_my_bag_of_words():
    words_to_index = {"hi": 0, "you": 1, "me": 2, "are": 3}
    examples = ["hi how are you", "hi hi hi you house"]
    answers = [[1, 1, 0, 1], [3, 1, 0, 0]]
    for ex, expected in zip(examples, answers):
        output = my_bag_of_words(ex, words_to_index)
        if (output != np.asarray(expected)).any():
            return f"Respuesta incorrecta: BOW('{ex}') = {output} != {expected}"
    return "¡Tests correctos!"


print(test_my_bag_of_words())

¡Tests correctos!


Ahora aplicamos la función anterior a todos los datos.

La representación *bag of words* devuelve vectores __*sparse*__ (la mayoría de sus entradas son ceros), con lo que conviene usar estructuras de datos especiales para datos *sparse* para ser eficientes.

Hay muchos [tipos de representación sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html), y `sklearn` sólo trabaja con la representación [csr matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix), que es la que usamos.

In [33]:
from scipy import sparse as sp_sparse

In [34]:
X_train_mybag = sp_sparse.vstack(
    [sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX)) for text in X_train]
)
X_val_mybag = sp_sparse.vstack(
    [sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX)) for text in X_val]
)
X_test_mybag = sp_sparse.vstack(
    [sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX)) for text in X_test]
)
print("X_train shape ", X_train_mybag.shape)
print("X_val shape ", X_val_mybag.shape)
print("X_test shape ", X_test_mybag.shape)

X_train shape  (85000, 5000)
X_val shape  (15000, 5000)
X_test shape  (30000, 5000)


## Preguntas finales

* ¿Qué efecto tienen los caracteres de `REPLACE_BY_SPACE` y `GOOD_CHARS` sobre las features generadas? ¿Se te ocurre una mejor elección de los caracteres escogidos?
* Como hemos comentado en el worksheet, la representación Bag of Words no tiene en cuenta el orden de los tokens. Pero hay extensiones de Bag of Words que en cierto grado tienen en cuenta el orden de las palabras. ¿Puedes dar una idea de cómo hacer esto? ¿Crees que puede ser relevante para el contexto del problema?
* ¿Cómo actuará un modelo entrenado con estas features ante erratas?

In [ ]:
# Respuestas finales

"""
1. REPLACE_BY_SPACE y GOOD_CHARS:
- REPLACE_BY_SPACE separa palabras unidas por caracteres especiales (/(){}[]|@,;)
- GOOD_CHARS filtra caracteres no deseados, manteniendo solo letras minúsculas, números y #+_
- Mejoraría incluyendo '-' y '.' para nombres de lenguajes y versiones

2. Extensión Bag of Words:
- Usaría n-gramas para capturar secuencias de palabras
- El orden es relevante pero no crítico para clasificación de etiquetas
- Ejemplo: "how to use java" vs "java how to use" tendrían diferentes bigramas

3. Comportamiento ante erratas:
- El modelo actual es sensible a errores ortográficos
- Implementaría:
  * Corrección ortográfica
  * Stemming
  * Fuzzy matching
"""

In [37]:
!pip install nltk textblob

import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/odremanferrer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/odremanferrer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
# Mejoras implementadas optimizadas:

# 1. Mejora de caracteres especiales
REPLACE_BY_SPACE = "[/(){}\\[\\]\\|@,;]"
GOOD_CHARS = ascii_lowercase + "".join([str(n) for n in range(10)]) + " #+_-."

# 2. Función mejorada de preprocesado con stemming
def text_prepare_improved(text: str) -> str:
    """
    Preprocesa el texto inicial:
    1. eliminando espacios al inicio y final, y convirtiéndolo a minúsculas
    2. cambia los caracteres de REPLACE_BY_SPACE por espacios
    3. elimina los caracteres que no estén en GOOD_CHARS
    4. elimina los tokens que sean STOPWORDS
    5. aplica stemming a las palabras

    text: str
    return: str
    """
    if not text:  # Si text es None o vacío
        return ""
        
    # 1. Eliminar espacios y convertir a minúsculas
    text = text.lower().strip()
    
    # 2. Reemplazar caracteres especiales por espacios
    text = re.sub(REPLACE_BY_SPACE, ' ', text)
    
    # 3. Eliminar caracteres que no estén en GOOD_CHARS
    text = ''.join([char for char in text if char in GOOD_CHARS])
    
    # 4. Eliminar stopwords y aplicar stemming
    tokens = text_tokenizer(text)
    tokens = [token for token in tokens if token not in STOPWORDS]
    
    # 5. Aplicar stemming
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    return ' '.join(tokens)

# 3. Función para crear n-gramas
def create_ngrams(text: str, n: int = 2) -> list[str]:
    """
    Crea n-gramas a partir de un texto tokenizado.
    
    text: str, texto a procesar
    n: int, tamaño de los n-gramas
    return: list[str], lista de n-gramas
    """
    tokens = text_tokenizer(text)
    ngrams = []
    for i in range(len(tokens) - n + 1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

# 4. Función mejorada de Bag of Words con n-gramas
def my_bag_of_words_improved(text: str, words_to_index: dict[str, int], ngram_size: int = 2) -> np.array:
    """
    text: str
    words_to_index: dict, diccionario con los índices del vocabulario
    ngram_size: int, tamaño de los n-gramas a considerar

    return
    result_vector: numpy.array, vector con la representación bag-of-words de `text`
    """
    dict_size = len(words_to_index)
    result_vector = np.zeros(dict_size)

    # Tokenizar el texto
    tokens = text_tokenizer(text)
    
    # Contar ocurrencias de cada palabra que esté en el vocabulario
    for token in tokens:
        if token in words_to_index:
            result_vector[words_to_index[token]] += 1
    
    # Añadir n-gramas
    ngrams = create_ngrams(text, ngram_size)
    for ngram in ngrams:
        if ngram in words_to_index:
            result_vector[words_to_index[ngram]] += 1

    return result_vector

# Ejemplo de uso:
X_train_improved = [text_prepare_improved(x) for x in X_train]
X_train_mybag_improved = sp_sparse.vstack(
    [sp_sparse.csr_matrix(my_bag_of_words_improved(text, WORDS_TO_INDEX)) for text in X_train_improved]
)

In [40]:
# Mostrar ejemplos del preprocesado
print("Ejemplos de preprocesado mejorado:")
for i in range(3):
    print(f"Original: {X_train[i]}")
    print(f"Mejorado: {text_prepare_improved(X_train[i])}")
    print("-" * 50)

# Mostrar algunos n-gramas generados
print("\nEjemplos de n-gramas:")
sample_text = X_train[0]
print(f"Texto original: {sample_text}")
print(f"Bigramas: {create_ngrams(sample_text, 2)}")
print(f"Trigramas: {create_ngrams(sample_text, 3)}")
print("-" * 50)

# Mostrar el efecto del stemming
print("\nEjemplos de stemming:")
sample_words = ["running", "jumped", "better", "programming"]
stemmer = nltk.stem.PorterStemmer()
for word in sample_words:
    print(f"{word} -> {stemmer.stem(word)}")

Ejemplos de preprocesado mejorado:
Original: create array leaf nodes html dom using javascript
Mejorado: creat array leaf node html dom use javascript
--------------------------------------------------
Original: make maven use test resources
Mejorado: make maven use test resourc
--------------------------------------------------
Original: get path user installed java application
Mejorado: get path user instal java applic
--------------------------------------------------

Ejemplos de n-gramas:
Texto original: create array leaf nodes html dom using javascript
Bigramas: ['create array', 'array leaf', 'leaf nodes', 'nodes html', 'html dom', 'dom using', 'using javascript']
Trigramas: ['create array leaf', 'array leaf nodes', 'leaf nodes html', 'nodes html dom', 'html dom using', 'dom using javascript']
--------------------------------------------------

Ejemplos de stemming:
running -> run
jumped -> jump
better -> better
programming -> program
